In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import TensorBoard
import os
import tempfile
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

NAME = "Covid-Detection-{}".format(int(time.time()))

### Importing and showing dataset from kaggle repository

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/UPC-IA-WV71-2021/IA_TF/main/Cleaned-Data.csv")
df.head()

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,Infected,Age_0-9,Age_10-19,Age_20-24,Age_25-59,Age_60+,Gender_Female,Gender_Male,Gender_Transgender,Severity_Mild,Severity_Moderate,Severity_None,Severity_Severe,Contact_Dont-Know,Contact_No,Contact_Yes,Country
0,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,China
1,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,China
2,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,China
3,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,China
4,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,China


### Prepare dataset for input layer

First, we need to get rid of the information that its not useful for detecting if a person is infected with coronavirus. This means, we need to modify the original dataset.

In [3]:
cols_to_drop = ['None_Sympton','Age_0-9','Age_10-19','Age_20-24','Age_25-59','Age_60+','Gender_Female','Gender_Male','Gender_Transgender','Severity_Mild','Severity_Moderate','Severity_None','Severity_Severe','Contact_Dont-Know','Contact_No','Contact_Yes','Country']
df = df.drop(cols_to_drop, axis = 1)
df.head()

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,Infected
0,1,1,1,1,1,1,1,1,1,0
1,1,1,1,1,1,1,1,1,1,0
2,1,1,1,1,1,1,1,1,1,0
3,1,1,1,1,1,1,1,1,1,0
4,1,1,1,1,1,1,1,1,1,0


We confirm that the data in the dataset now has only values that will be used for training.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316800 entries, 0 to 316799
Data columns (total 10 columns):
 #   Column                   Non-Null Count   Dtype
---  ------                   --------------   -----
 0   Fever                    316800 non-null  int64
 1   Tiredness                316800 non-null  int64
 2   Dry-Cough                316800 non-null  int64
 3   Difficulty-in-Breathing  316800 non-null  int64
 4   Sore-Throat              316800 non-null  int64
 5   Pains                    316800 non-null  int64
 6   Nasal-Congestion         316800 non-null  int64
 7   Runny-Nose               316800 non-null  int64
 8   Diarrhea                 316800 non-null  int64
 9   Infected                 316800 non-null  int64
dtypes: int64(10)
memory usage: 24.2 MB


To analize the dataset this is converted into an array

In [5]:
dataset = df.values
dataset

array([[1, 1, 1, ..., 1, 1, 0],
       [1, 1, 1, ..., 1, 1, 0],
       [1, 1, 1, ..., 1, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

Here we define the metrics of the coordinate axes. Notice that the all rows in the dataset are asigned to "X", but we only extract the first 8 columns since those are the columns that are being analized in the future. Then, the column "Infected" is asigned to the Y axis.

In [6]:
X = dataset[: ,0:8]
Y = dataset[: ,9]

We split the dataset into a test set, validation set and training set. The size of the validation and test will be 30% of the dataset. Then, we split again the validation set and test set so they have the same size.

Now, we have six variables that will be used in the dataset:

*   X_train (70% of dataset, 5 input features)
*   X_val (15% of dataset, 5 input features)
*   X_test (15% of dataset, 5 input features)
*   Y_train (70% of dataset, 1 label features)
*   Y_val (15% of dataset, 1 label features)
*   Y_test (15% of dataset, 1 label features)

In [7]:
X_train, X_val_test, Y_train, Y_val_test = train_test_split(X, Y, test_size=0.3)

In [8]:
X_val, X_test, Y_val, Y_test = train_test_split(X_val_test, Y_val_test, test_size=0.5)

This shows the shapes of the arrays (i.e. their dimensions)

In [9]:
print(X_train.shape, X_val.shape, X_test.shape, Y_train.shape, Y_val.shape, Y_test.shape)

(221760, 8) (47520, 8) (47520, 8) (221760,) (47520,) (47520,)


The training set has 221760 data points, the validation set has 47520 data points and the test set has 47520 data points. This is because the amount of data in the dataset is odd. The X variables have 8 input features, while the Y variables has one feature (the one to predict if a person has coronavirus).

###CNN

####Setting up the Architecture

In [11]:
model = Sequential([ #the model will be stored in the variable and it will be described sequentially
    Dense(32, activation='relu', input_shape=(8,)), #first dense layer with 32 neurons, RelU activation and 5 inputs
    Dense(32, activation='relu'), #second dense layer with 32 neurons and RelU activation
    Dense(1, activation='sigmoid'), #third dense layer with 1 neuron and sigmoid activation 
])

In [12]:
model.compile(optimizer='sgd', #using Stochastic Gradient Descent algorithm
              loss='binary_crossentropy', #loss function for outputs (takes values 1 or 0)
              metrics=['accuracy']) #tracks the accuracy

In [13]:
h = model.fit(X_train, Y_train,
          batch_size=32, epochs=100,  
          validation_data=(X_val, Y_val))

Epoch 1/100
6930/6930 [==============================] - 11s 1ms/step - loss: 0.1503 - accuracy: 0.9088 - val_loss: 0.1275 - val_accuracy: 0.9090
Epoch 2/100
6930/6930 [==============================] - 9s 1ms/step - loss: 0.1269 - accuracy: 0.9088 - val_loss: 0.1264 - val_accuracy: 0.9095
Epoch 3/100
6930/6930 [==============================] - 9s 1ms/step - loss: 0.1264 - accuracy: 0.9086 - val_loss: 0.1261 - val_accuracy: 0.9079
Epoch 4/100
6930/6930 [==============================] - 10s 1ms/step - loss: 0.1263 - accuracy: 0.9095 - val_loss: 0.1260 - val_accuracy: 0.9082
Epoch 5/100
6930/6930 [==============================] - 10s 1ms/step - loss: 0.1262 - accuracy: 0.9088 - val_loss: 0.1259 - val_accuracy: 0.9073
Epoch 6/100
6930/6930 [==============================] - 9s 1ms/step - loss: 0.1261 - accuracy: 0.9095 - val_loss: 0.1260 - val_accuracy: 0.9073
Epoch 7/100
6930/6930 [==============================] - 9s 1ms/step - loss: 0.1261 - accuracy: 0.9092 - val_loss: 0.1259 - val

In [14]:
model.evaluate(X_test, Y_test)[1] #index 1 returns accuracy, index 0 returns loss

1485/1485 [==============================] - 1s 991us/step - loss: 0.1272 - accuracy: 0.9076


0.9075546860694885